# revelance ranking

In [1]:
import pandas as pd
from tqdm import tqdm
import pickle
tqdm.pandas()

In [2]:
import json

with open('C:/Users/ngoph/Desktop/zalo_ai/wikipedia_20220620_cleaned/wikipedia_20220620_cleaned.jsonl', 'r',encoding='utf-8') as json_file:
    json_list = list(json_file)

In [3]:
json_list = [json.loads(json_str) for json_str in json_list]

In [4]:
df = pd.DataFrame(json_list)
df.head()

,id,url,title,text,timestamp,revid
0,2,https://vi.wikipedia.org/wiki?curid=2,Trang Chính,"Trang Chính\n\n<templatestyles src=""Wiki2021/s...",2022-05-12T12:46:53Z,68591979
1,4,https://vi.wikipedia.org/wiki?curid=4,Internet Society,Internet Society\n\nInternet Society hay ISOC ...,2022-01-20T07:59:10Z,67988747
2,13,https://vi.wikipedia.org/wiki?curid=13,Tiếng Việt,"Tiếng Việt\n\nTiếng Việt, cũng gọi là tiếng Vi...",2022-05-29T03:42:42Z,68660631
3,24,https://vi.wikipedia.org/wiki?curid=24,Ohio,"Ohio\n\nOhio (viết tắt là OH, viết tắt cũ là O...",2022-04-17T08:15:22Z,68482118
4,26,https://vi.wikipedia.org/wiki?curid=26,California,"California\n\nCalifornia (phát âm như ""Ca-li-p...",2022-06-16T15:27:07Z,68738039


In [5]:
# free up memory
del json_list

In [6]:
from bs4 import BeautifulSoup
import re

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def normalize_text(text):
    text = strip_html_tags(text)
    # remove \n 
    text = re.sub(r'\n', ' ', text)
    # remove \xa0
    text = re.sub(r'\xa0', ' ', text)
    # remove 'BULLET::::-'
    text = re.sub(r'BULLET::::-', ' ', text)
    # remove duplicate spaces
    text = re.sub(r'\s+', ' ', text)
    return text

In [7]:
df['text'] = df['text'].progress_apply(normalize_text)

  0%|          | 378/1273469 [00:00<22:23, 947.61it/s]c:\Users\ngoph\.conda\envs\research\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
  0%|          | 4732/1273469 [00:02<09:17, 2274.01it/s]c:\Users\ngoph\.conda\envs\research\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
100%|██████████| 1273469/1273469 [01:27<00:00, 14603.83it/s]


In [8]:
df.head()

,id,url,title,text,timestamp,revid
0,2,https://vi.wikipedia.org/wiki?curid=2,Trang Chính,Trang Chính __NOEDITSECTION__,2022-05-12T12:46:53Z,68591979
1,4,https://vi.wikipedia.org/wiki?curid=4,Internet Society,Internet Society Internet Society hay ISOC là ...,2022-01-20T07:59:10Z,67988747
2,13,https://vi.wikipedia.org/wiki?curid=13,Tiếng Việt,"Tiếng Việt Tiếng Việt, cũng gọi là tiếng Việt ...",2022-05-29T03:42:42Z,68660631
3,24,https://vi.wikipedia.org/wiki?curid=24,Ohio,"Ohio Ohio (viết tắt là OH, viết tắt cũ là O.) ...",2022-04-17T08:15:22Z,68482118
4,26,https://vi.wikipedia.org/wiki?curid=26,California,"California California (phát âm như ""Ca-li-phót...",2022-06-16T15:27:07Z,68738039


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
corpus = df['text'].tolist()

In [ ]:
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(corpus)
# filename = 'tfidfVectorizer.pkl'
# pickle.dump(vectorizer, open(filename, 'wb'))

In [ ]:
# use this code to load
vectorizer = pickle.load(open('tfidfVectorizer.pkl', 'rb'))
X = vectorizer.transform(corpus)

In [12]:
vectorizer.get_feature_names_out()

array(['00', '000', '0000', ..., '𪤄葛鐄', '𪼀越南', '𬇕尾'], dtype=object)

# test 

In [84]:
question = 'Ai là người phát minh ra máy bay?'
q = vectorizer.transform([question])

# get top 10 similar documents
top_k = 10
top_k_idx = X.dot(q.T).toarray().flatten().argsort()[-top_k:][::-1]
top_k_idx

array([1143961,  578863,  582928,   25801,   16864, 1244663, 1256162,
         10774, 1233222,  460583], dtype=int64)

In [85]:
df.iloc[top_k_idx]

,id,url,title,text,timestamp,revid
1143961,3523602,https://vi.wikipedia.org/wiki?curid=3523602,Máy bay dân dụng,"Máy bay dân dụng Một máy bay dân dụng, máy bay...",2022-05-11T05:00:55Z,68587135
578863,1684926,https://vi.wikipedia.org/wiki?curid=1684926,Danh sách máy bay quân sự giữa hai cuộc chiến ...,Danh sách máy bay quân sự giữa hai cuộc chiến ...,2016-09-07T08:12:10Z,24183214
582928,1739768,https://vi.wikipedia.org/wiki?curid=1739768,Danh sách máy bay ném bom,Danh sách máy bay ném bom Danh sách máy bay né...,2021-10-19T06:38:50Z,66411836
25801,141312,https://vi.wikipedia.org/wiki?curid=141312,Máy bay quân sự,Máy bay quân sự Một máy bay quân sự là bất kỳ ...,2021-12-20T12:11:51Z,67692885
16864,88610,https://vi.wikipedia.org/wiki?curid=88610,Polikarpov,Polikarpov Polikarpov là một phòng thiết kế (O...,2022-03-11T06:44:10Z,68265694
1244663,19433483,https://vi.wikipedia.org/wiki?curid=19433483,Chuyến bay 66 của Carson Air,Chuyến bay 66 của Carson Air Carson Air Flight...,2021-12-26T17:26:15Z,67741329
1256162,19556717,https://vi.wikipedia.org/wiki?curid=19556717,Chuyến bay 328 của United Airlines,Chuyến bay 328 của United Airlines Chuyến bay ...,2022-02-08T00:38:56Z,68118518
10774,59155,https://vi.wikipedia.org/wiki?curid=59155,Máy bay cánh cố định,Máy bay cánh cố định Máy bay có cánh cố định l...,2021-10-17T10:19:10Z,66383065
1233222,17298068,https://vi.wikipedia.org/wiki?curid=17298068,Chế độ máy bay,Chế độ máy bay Chế độ máy bay có thể là: Chế đ...,2021-07-13T05:28:06Z,65200799
460583,1401009,https://vi.wikipedia.org/wiki?curid=1401009,Danh sách máy bay trong Chiến tranh thế giới t...,Danh sách máy bay trong Chiến tranh thế giới t...,2022-06-02T12:03:32Z,68676628


In [81]:
from transformers import pipeline
qa = pipeline('question-answering', model='model_2\saved',
    device=0
)

In [82]:
import tensorflow as tf
model_1 = tf.keras.models.load_model('model_1')
LABEL={
	'wiki':0,
	'number':1,
	'text':2
}

In [86]:
inputs = []
for i in top_k_idx:
    row = df.iloc[i]
    context = row['text']
    inputs += [{'question': question, 'context': context}]

outputs = qa(inputs)
print(question)
type=model_1.predict([question],verbose=0).argmax()
print(list(LABEL.keys())[type])
for i,output in enumerate(outputs):
    print(df.iloc[top_k_idx[i]].title)
    print(output)
    print('----')

Ai là người phát minh ra máy bay?
wiki
Máy bay dân dụng
{'score': 0.9873418211936951, 'start': 342, 'end': 363, 'answer': 'hai anh em nhà Wright'}
----
Danh sách máy bay quân sự giữa hai cuộc chiến tranh thế giới
{'score': 5.980842904262929e-11, 'start': 5795, 'end': 5810, 'answer': 'Albatros Al 101'}
----
Danh sách máy bay ném bom
{'score': 1.4204627518665802e-07, 'start': 95, 'end': 110, 'answer': 'Máy bay ném bom'}
----
Máy bay quân sự
{'score': 6.571782007469551e-11, 'start': 0, 'end': 15, 'answer': 'Máy bay quân sự'}
----
Polikarpov
{'score': 7.887290820463022e-09, 'start': 108, 'end': 138, 'answer': 'Nikolai Nikolaevich Polikarpov'}
----
Chuyến bay 66 của Carson Air
{'score': 0.0010760389268398285, 'start': 1005, 'end': 1018, 'answer': 'Robert Brandt'}
----
Chuyến bay 328 của United Airlines
{'score': 6.243306652914393e-13, 'start': 1095, 'end': 1109, 'answer': 'Boeing 777-200'}
----
Máy bay cánh cố định
{'score': 2.0618230701322654e-13, 'start': 252, 'end': 272, 'answer': 'máy b

# inference

In [18]:
with open('data/zac2022_testa_only_question.json',encoding='utf-8') as f:
    data=json.load(f)
    reference_df = pd.json_normalize(data,'data')

In [89]:
submission = []

for row in tqdm(reference_df.iterrows(),total=len(reference_df)):
    question = row[1]['question']
    q = vectorizer.transform([question])
    top_k = 3
    top_k_idx = X.dot(q.T).toarray().flatten().argsort()[-top_k:][::-1]
    result = []
    for i in top_k_idx:
        doc_row = df.iloc[i]
        context = doc_row['text']
        result.append((doc_row['title'], qa(question=question, context=context)))

    result.sort(key=lambda x: x[1]['score'], reverse=True)
    best_result = result[0]
    type=model_1.predict([question],verbose=0).argmax()
    if type==LABEL['wiki']:
        answer='wiki/'+best_result[1]['answer'].replace(' ','_')
    else:
        answer=best_result[1]['answer']
        
    if best_result[1]['score']<1e-8:
        answer=None

    submission.append({
        'id': row[1]['id'],
        'question': question,
        'answer': answer
    })

  0%|          | 2/600 [00:02<11:15,  1.13s/it]c:\Users\ngoph\.conda\envs\research\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 600/600 [08:19<00:00,  1.20it/s]


In [1]:
df_submission = pd.DataFrame(submission)

NameError: name 'pd' is not defined

In [91]:
json_data = df_submission.to_dict(orient='records')
with open('submission.json', 'w', encoding='utf-8') as f:
    json.dump({'data': json_data}, f, ensure_ascii=False, indent=2)